In [17]:
import pickle

file_path = '/Users/st.george/Desktop/1_3_10_testing_good.pkl'
with open(file_path, 'rb') as file:
    data = pickle.load(file)

In [45]:
#alpha & beta
import yfinance as yf
import statsmodels.api as sm

sp500 = yf.download('^GSPC', start='2024-01-02', end='2024-11-01', progress=False)

sp500['Daily Returns'] = sp500['Adj Close'].pct_change().fillna(0)

benchmark_returns = sp500['Daily Returns']
daily_returns.index = benchmark_returns.index


aligned_returns = pd.concat([daily_returns, benchmark_returns], axis=1).dropna()

portfolio_returns = aligned_returns.iloc[:, 0]
benchmark_returns = aligned_returns.iloc[:, 1]

X = sm.add_constant(benchmark_returns)  

model = sm.OLS(portfolio_returns, X).fit()

alpha = model.params['const'] * 252  
beta = model.params[benchmark_returns.name]  

In [47]:
import pandas as pd
import numpy as np

daily_pnl = data['daily_pnl']
daily_pnl_series = pd.Series(daily_pnl)

initial_portfolio_value = 100000000  
daily_returns = daily_pnl_series / initial_portfolio_value

cumulative_returns = (1 + daily_returns).cumprod()

def annualized_return(returns):
    return (1 + returns.mean()) ** 252 - 1

def annualized_volatility(returns):
    return returns.std() * np.sqrt(252)

def sharpe_ratio(returns, risk_free_rate=0.05):
    excess_returns = returns - risk_free_rate / 252
    return annualized_return(excess_returns) / annualized_volatility(returns)

def maximum_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return drawdown.min()

def value_at_risk(returns, level=0.05):
    return np.percentile(returns, level * 100)

def sortino_ratio(returns, risk_free_rate=0.03):
    downside_returns = returns[returns < 0]
    excess_returns = returns - risk_free_rate / 252
    return annualized_return(excess_returns) / (downside_returns.std() * np.sqrt(252))

def skewness(returns):
    return returns.skew()

def kurtosis(returns):
    return returns.kurt()

metrics = {
    "Annualized Return": annualized_return(daily_returns),
    "Annualized Volatility": annualized_volatility(daily_returns),
    "Sharpe Ratio": sharpe_ratio(daily_returns),
    "Maximum Drawdown": maximum_drawdown(daily_returns),
    "95% Value at Risk": value_at_risk(daily_returns),
    "Sortino Ratio": sortino_ratio(daily_returns),
    "Skewness": skewness(daily_returns),
    "Excess Kurtosis": kurtosis(daily_returns),
    "Alpha": alpha,
    "Beta": beta
}

metrics


{'Annualized Return': 0.16175989516537959,
 'Annualized Volatility': 0.0677319719354648,
 'Sharpe Ratio': 1.5521117139481466,
 'Maximum Drawdown': -0.035903252099580396,
 '95% Value at Risk': -0.006842473222746863,
 'Sortino Ratio': 2.6942350643985686,
 'Skewness': 0.08449200518750445,
 'Excess Kurtosis': 2.4273546564100745,
 'Alpha': 0.1535929729245607,
 'Beta': -0.015803722958610728}

In [59]:
measurement = pd.DataFrame(metrics, index = [0]).T
print(measurement)

                              0
Annualized Return      0.161760
Annualized Volatility  0.067732
Sharpe Ratio           1.552112
Maximum Drawdown      -0.035903
95% Value at Risk     -0.006842
Sortino Ratio          2.694235
Skewness               0.084492
Excess Kurtosis        2.427355
Alpha                  0.153593
Beta                  -0.015804


In [65]:
measurement.to_excel("/Users/st.george/Desktop/measurement.xlsx")